In [3]:
import os
import pathlib

In [4]:
#print(pathlib.Path.cwd().parts)

In [5]:
#if "models-master" in pathlib.Path.cwd().parts:
#  while "models-master" in pathlib.Path.cwd().parts:
#    os.chdir('..') # change the directory
#elif not pathlib.Path('models').exists():
#  !git clone --depth 1 https://github.com/tensorflow/models

Compile protobufs and install the object_detection package

In [6]:
#%%bash
#cd models-master/research/
#protoc object_detection/protos/*.proto --python_out=.

In [7]:
#%%bash 
#cd models-master/research
#pip install .

### Imports

In [8]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
from IPython.display import display

Import the object detection module.

In [9]:
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

Patches:

In [10]:
# patch tf1 into `utils.ops`
utils_ops.tf = tf.compat.v1

# Patch the location of gfile
tf.gfile = tf.io.gfile

# Model preparation 

## Variables

Any model exported using the `export_inference_graph.py` tool can be loaded here simply by changing the path.

By default we use an "SSD with Mobilenet" model here. See the [detection model zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md) for a list of other models that can be run out-of-the-box with varying speeds and accuracies.

## Loader

In [1]:
import readmodel
model = readmodel.modelchoose( 'model_1', (256, 256, 3), 6)
model.summary()

last change
X current model is: model_1_2CCP_1FDD
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 256, 256, 32)      896       
_________________________________________________________________
activation (Activation)      (None, 256, 256, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 254, 254, 32)      9248      
_________________________________________________________________
activation_1 (Activation)    (None, 254, 254, 32)      0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 127, 127, 32)      0         
_________________________________________________________________
dropout (Dropout)            (None, 127, 127, 32)      0         
_________________________________________________________________
conv2d

In [ ]:
from tensorflow.keras.models import load_model
#from tensorflow import keras
#import keras
# returns a compiled model
# identical to the previous one
#model = load_model('my_model.h5')
load_path = '/home/ali/my_project/gaps/saved_models/'
model_name = 'model_1_epochs_5_batchsize_50.h5'
model = load_model(load_path + model_name)

In [27]:
def load_model(model_name):
  base_url = 'http://download.tensorflow.org/models/object_detection/'
  model_file = model_name + '.tar.gz'
  model_dir = tf.keras.utils.get_file(
    fname=model_name, 
    origin=base_url + model_file,
    untar=True)

  model_dir = pathlib.Path(model_dir)/"saved_model"

  model = tf.saved_model.load(str(model_dir))
  model = model.signatures['serving_default']

  return model

## Loading label map
Label maps map indices to category names, so that when our convolution network predicts `5`, we know that this corresponds to `airplane`.  Here we use internal utility functions, but anything that returns a dictionary mapping integers to appropriate string labels would be fine

In [11]:
# List of the strings that is used to add correct label for each box.
# List of the strings that is used to add correct label for each box.
OBJDIR = '/home/ali/my_project/models-master/research/object_detection'
PATH_TO_LABELS = OBJDIR + '/data/mscoco_label_map.pbtxt'
#PATH_TO_LABELS = 'models/research/object_detection/data/mscoco_label_map.pbtxt'
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

In [16]:
def load_my_label():
    pass # if there is any

category_index = {1: {'id': 1, 'name': 'person'},
                  2: {'id': 2, 'name': 'bicycle'},
                  3: {'id': 3, 'name': 'car'},
                  4: {'id': 4, 'name': 'motorcycle'},
                  5: {'id': 5, 'name': 'airplane'},
                  6: {'id': 6, 'name': 'bus'}}

In [17]:
print(category_index)

{1: {'id': 1, 'name': 'person'}, 2: {'id': 2, 'name': 'bicycle'}, 3: {'id': 3, 'name': 'car'}, 4: {'id': 4, 'name': 'motorcycle'}, 5: {'id': 5, 'name': 'airplane'}, 6: {'id': 6, 'name': 'bus'}}


For the sake of simplicity we will test on 2 images:

In [18]:
print(OBJDIR)

/home/ali/my_project/models-master/research/object_detection


In [19]:
# If you want to test the code with your images, just add path to the images to the TEST_IMAGE_PATHS.
#PATH_TO_TEST_IMAGES_DIR = OBJDIR + '/test_images'
#PATH_TO_TEST_IMAGES_DIR = pathlib.Path('models/research/object_detection/test_images')
PATH_TO_TEST_IMAGES_DIR = pathlib.Path('/home/ali/my_project/models-master/research/object_detection/test_images')
#PATH_TO_TEST_IMAGES_DIR = 


In [20]:
print(PATH_TO_TEST_IMAGES_DIR.glob("*.jpg"))

<generator object Path.glob at 0x7f79bc324250>


In [21]:
TEST_IMAGE_PATHS = sorted(list(PATH_TO_TEST_IMAGES_DIR.glob("*.jpg")))

In [22]:
print(TEST_IMAGE_PATHS)

[PosixPath('/home/ali/my_project/models-master/research/object_detection/test_images/image1.jpg'), PosixPath('/home/ali/my_project/models-master/research/object_detection/test_images/image2.jpg')]


# Detection

Load an object detection model:

In [26]:
model_name = 'ssd_mobilenet_v1_coco_2017_11_17'
detection_model = load_model(model_name)

NameError: name 'load_model' is not defined

In [23]:
detection_model = model

Check the model's input signature, it expects a batch of 3-color images of type uint8: 

In [24]:
print(detection_model.inputs)

[<tf.Tensor 'conv2d_input:0' shape=(None, 256, 256, 3) dtype=float32>]


And retuns several outputs:

In [ ]:
detection_model.output_dtypes

In [31]:
detection_model.output_shapes

AttributeError: 'Sequential' object has no attribute 'output_shapes'

Add a wrapper function to call the model, and cleanup the outputs:

In [28]:
def run_inference_for_single_image(model, image):
  image = np.asarray(image)
  # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
  input_tensor = tf.convert_to_tensor(image)
  # The model expects a batch of images, so add an axis with `tf.newaxis`.
  input_tensor = input_tensor[tf.newaxis,...]

  # Run inference
  output_dict = model(input_tensor)

  # All outputs are batches tensors.
  # Convert to numpy arrays, and take index [0] to remove the batch dimension.
  # We're only interested in the first num_detections.
  num_detections = int(output_dict.pop('num_detections'))
  output_dict = {key:value[0, :num_detections].numpy() 
                 for key,value in output_dict.items()}
  output_dict['num_detections'] = num_detections

  # detection_classes should be ints.
  output_dict['detection_classes'] = output_dict['detection_classes'].astype(np.int64)
   
  # Handle models with masks:
  if 'detection_masks' in output_dict:
    # Reframe the the bbox mask to the image size.
    detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
              output_dict['detection_masks'], output_dict['detection_boxes'],
               image.shape[0], image.shape[1])      
    detection_masks_reframed = tf.cast(detection_masks_reframed > 0.5,
                                       tf.uint8)
    output_dict['detection_masks_reframed'] = detection_masks_reframed.numpy()
    
  return output_dict

Run it on each test image and show the results:

In [29]:
def show_inference(model, image_path):
  # the array based representation of the image will be used later in order to prepare the
  # result image with boxes and labels on it.
  image_np = np.array(Image.open(image_path))
  # Actual detection.
  output_dict = run_inference_for_single_image(model, image_np)
  # Visualization of the results of a detection.
  vis_util.visualize_boxes_and_labels_on_image_array(
      image_np,
      output_dict['detection_boxes'],
      output_dict['detection_classes'],
      output_dict['detection_scores'],
      category_index,
      instance_masks=output_dict.get('detection_masks_reframed', None),
      use_normalized_coordinates=True,
      line_thickness=8)

  display(Image.fromarray(image_np))

In [30]:
for image_path in TEST_IMAGE_PATHS:
  show_inference(detection_model, image_path)


ValueError: Incompatible type conversion requested to type 'uint8' for variable of type 'float32'

## Instance Segmentation

In [ ]:
model_name = "mask_rcnn_inception_resnet_v2_atrous_coco_2018_01_28"
masking_model = load_model("mask_rcnn_inception_resnet_v2_atrous_coco_2018_01_28")

The instance segmentation model includes a `detection_masks` output:

In [ ]:
masking_model.output_shapes

In [ ]:
for image_path in TEST_IMAGE_PATHS:
  show_inference(masking_model, image_path)

In [ ]:
TEST_IMAGE_PATHS